In [1]:
import pandas as pd
from math import isnan

In [2]:
GT_plates = pd.read_excel("PlatesLateral.ods", engine="odf")
GT_plates_videos = pd.read_excel("PlatesVideos.ods", engine="odf")
results = pd.read_csv('../out/results_confosable_Bidi.csv')
results_videos = pd.read_csv('../out/results_videos.csv')
GT_plates = GT_plates[GT_plates['Plate'].notna()]
GT_plates['Plate'] = GT_plates['Plate'].apply(lambda x: str(x).replace('“','').replace('”',''))
GT_plates

,Model,Video,Plate,Init_frame,End_frame,Detected
0,a109,a109e_takeoff,JA6801,NaN,NaN,Y
1,a109,augusta_a109,D-HAVN,NaN,NaN,Y
2,a109,aw109sp,S2-AJP,NaN,NaN,Y
3,a109,corporate,N433AK,NaN,NaN,Y
4,a109,crashed,N109EX,NaN,NaN,Y
...,...,...,...,...,...,...
132,uh1,huey,VH-UYH,57.0,113.0,Y
133,uh1,uh1,N617WC,NaN,NaN,Y
134,w3,EHS,0718,NaN,NaN,Y
135,w3,Rare,SN-33XG,NaN,NaN,Y


In [3]:
results

,Model,Video,Frame,Pred_plate,Confidence
0,a109,a109e_takeoff,0,NaN,0.000000
1,a109,a109e_takeoff,1,NaN,0.000000
2,a109,a109e_takeoff,2,405,0.680593
3,a109,a109e_takeoff,3,NaN,0.000000
4,a109,a109e_takeoff,4,NaN,0.000000
...,...,...,...,...,...
4097,w3,Saving,102,NaN,0.000000
4098,w3,Saving,103,NaN,0.000000
4099,w3,Saving,104,NaN,0.000000
4100,w3,Saving,105,NaN,0.000000


In [4]:
results_videos

,Model,Video,Frame,Pred_plate,Confidence
0,a109,corporate,0,W2SHDT,0.999998
1,a109,corporate,1,N2&SHD,1.000000
2,a109,corporate,2,M2SHD,0.999951
3,a109,corporate,3,N2UHD,1.000000
4,a109,corporate,4,NAA,0.021489
...,...,...,...,...,...
76441,w3,Rare,325,$0,0.854689
76442,w3,Rare,326,0,0.630538
76443,w3,Rare,327,1U,0.972253
76444,w3,Rare,328,SImS,0.988457


In [5]:
# Analyze results from images

In [6]:
n = GT_plates.shape[0]
plates_found = 0
plates_found_by_voting = 0
detected = []
for i,row in GT_plates.iterrows():
    #Get plates recognized in that video
    preds = results.loc[(results['Model'] == row['Model']) & (results['Video'] == row['Video'])]
    #If a frame interval is defined, get only plates in that intervals
    if not isnan(row['Init_frame']):
        preds = preds[preds['Frame'].isin(range(int(row['Init_frame']),int(row['End_frame']+1)))]
    #Check if plate was found
    #print(row['Plate'])
    #print(list(preds['Pred_plate']))
    if row['Plate'] in list(preds['Pred_plate']) or str(row['Plate']).replace('-','') in list(preds['Pred_plate']):
        plates_found+=1
        detected+=['Y']
    else:
        detected+=['N']
    if len(preds['Pred_plate'].mode(dropna=True)):
        democracy_plate = preds['Pred_plate'].mode(dropna=True)[0]
        if democracy_plate == row['Plate']:
            plates_found_by_voting+=1
GT_plates['Detected'] = detected
GT_plates.to_excel("PlatesLateral.ods", engine="odf",index=False)
print(plates_found/n,plates_found_by_voting/n)
GT_plates

0.6058394160583942 0.5036496350364964


,Model,Video,Plate,Init_frame,End_frame,Detected
0,a109,a109e_takeoff,JA6801,NaN,NaN,Y
1,a109,augusta_a109,D-HAVN,NaN,NaN,Y
2,a109,aw109sp,S2-AJP,NaN,NaN,Y
3,a109,corporate,N433AK,NaN,NaN,Y
4,a109,crashed,N109EX,NaN,NaN,Y
...,...,...,...,...,...,...
132,uh1,huey,VH-UYH,57.0,113.0,Y
133,uh1,uh1,N617WC,NaN,NaN,Y
134,w3,EHS,0718,NaN,NaN,Y
135,w3,Rare,SN-33XG,NaN,NaN,Y


In [7]:
# Analyze results from videos

In [8]:
n = GT_plates_videos.shape[0]
plates_found = 0
plates_found_by_voting = 0
detected = []
for i,row in GT_plates_videos.iterrows():
    #Get plates recognized in that video
    preds = results_videos.loc[(results_videos['Model'] == row['Model']) & (results_videos['Video'] == row['Video'])]
    #Check if plate was found
    #print(row['Plate'])
    #print(list(preds['Pred_plate']))
    if row['Plate'] in list(preds['Pred_plate']) or str(row['Plate']).replace('-','') in list(preds['Pred_plate']):
        plates_found+=1
        detected+=['Y']
    else:
        detected+=['N']
    if len(preds['Pred_plate'].mode(dropna=True)):
        democracy_plate = preds['Pred_plate'].mode(dropna=True)[0]
        print(preds['Pred_plate'].mode(dropna=True))
        if democracy_plate == row['Plate']:
            plates_found_by_voting+=1
GT_plates_videos['Detected'] = detected
GT_plates_videos.to_excel("PlatesVideos.ods", engine="odf",index=False)
print(plates_found/n,plates_found_by_voting/n)
GT_plates_videos

0    GAINER
dtype: object
0    D-HANN
dtype: object
0    S2-AJP
dtype: object
0    N33AK
dtype: object
0        PARKING
1    TRESPASSING
dtype: object
0    MEROPORTO
dtype: object
0    MEROPORTO
dtype: object
0    MEROPORTO
dtype: object
0    MEROPORTO
dtype: object
0    EDMISTON
dtype: object
0    G-FUFU
dtype: object
0    2012-06
dtype: object
0    D-HARI
dtype: object
0    L-HDPLANESPOTT
dtype: object
0    2011
dtype: object
0    HAWTC
dtype: object
0    NTSL
dtype: object
0    RESCUE
dtype: object
0    NISSY
dtype: object
0    N318PD
dtype: object
0    AVIO
dtype: object
0    -PMK
dtype: object
0    N58IB
dtype: object
0    HEHDAVIATIO
dtype: object
0    HEHDAVIATIO
dtype: object
0    HEHDAVIATIO
dtype: object
0    ADU
dtype: object
0    NO25TG
dtype: object
0    HA-HBS
dtype: object
0    -KAE
dtype: object
0    HA-HBY
dtype: object
0    4-80B
dtype: object
0    G-BYZA
dtype: object
0    N253WC
dtype: object
0    -GMBL
1    CMABL
2    GMABL
3     GMBL
4     GMRL
5       SE
dtype: o

,Model,Video,Plate,Detected
0,a109,a109e_takeoff,JA6801,N
1,a109,augusta_a109,D-HAVN,Y
2,a109,aw109sp,S2-AJP,Y
3,a109,corporate,N433AK,Y
4,a109,crashed,N109EX,N
...,...,...,...,...
122,uh1,huey,VH-UYH,Y
123,uh1,uh1,N617WC,N
124,w3,EHS,0718,Y
125,w3,Rare,SN-33XG,Y
